In [15]:
import re
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import data
import config
import time
from dateparser.search import search_dates
import pandas as pd

stop_word = ['?', '.']
def clearing_word(word):
    word = re.sub('\x8e', 'é', word)
    word = re.sub('\x88', 'à', word)
    word = re.sub('\x9d', 'ù', word)
    word = re.sub('\x8f', 'è', word)
    word = re.sub('\x9e', 'û', word)
    word = re.sub('\x90', 'ê', word)
    word = re.sub('\x99', 'ô', word)
    word = re.sub('\x94', 'î', word)
   # word = re.sub('\x8f', 'è', word)
    word = re.sub('\x8d', 'ç', word)
    word = re.sub('õ', '', word)
    word = re.sub('Ê', '', word)
    word = re.sub('[?,.,!, \,,  %]', '', word)
    if word == 'û' or word == 'v' or word == 'é':
        word = ''
    if word =="2017êles":
        word = "2017"
    if "ênox" in word:
        word ="nox"
    return word

def clearing(pharse):
    """
    Arg: Data is a list of questions or answers
    Return: clean questions et answers 
    """
    #clean_data = []
    pharses =[]
    for word in pharse.strip().lower().split(' '):
       # line= data[i].lower().split(' ')        
        #for word in line:
        word = clearing_word(word)
        if word not in stop_word and len(word) !=0:       
            pharses.append(word)        
    return ' '.join(pharses)

def prepareData(PAIRS):
    pairs_trains, pairs_tests = [], []
   # PAIRS.extend(pairs1)
    index_train = random.sample(range(len(PAIRS)), int(len(PAIRS)*0.80))
    for i in range(len(PAIRS)):
        if i in index_train:
            pairs_trains.append(PAIRS[i])
        else:
            pairs_tests.append(PAIRS[i])
    print("Read %s sentence pairs of training set" % len(pairs_trains))
    print("Read %s sentence pairs of test set" % len(pairs_tests))
    return pairs_trains, pairs_tests

def get_all_convos():
    convos = []
    file0 = 'convos27juin.txt'
    #file1 = 'chatbot_tout_corpus.txt'
    #file2 = 'chatbot_tout_corpus_13juin.txt'
    #file3 = 'convos_120_nettoye.txt' # cette fichier viens d'etre ajouter
    liste_file = [file0]
    for file in liste_file:
        with open(file) as f:
            i=0
            for line in f:
                if i%2==0:
                    question = clearing(line)
                    if '++++' in question:
                         question = question[9:]
                else:
                    answer = clearing(line)
                    if '++++' in answer:
                        answer = answer[9:]
                    convos.append([question, answer])
                i+=1
        f.close()
    return convos

def simulation_pairs():
    DIRECT_VARIABLE = ['vitesse', 'batterie', 'position', 'km']
    TIME = []
    GEOGRAPHY = ['Lyon', 'hors lyon', 'paris', 'l\'étranger','france' , 'hors la france']
    BORDE = {'MAX': ['plus grand', 'plus grande', 'plus grands', 
         'plus grandes',  'moin faible', 'moins faibles',
         'plus vite', 'max', 'maximum', 'maximal', 'maximaux', 'maximale',
        'plus haut', 'plus haute', 'plus hautes', 'plus hauts',  
        'plus élevé', 'plus élevée', 'plus élevés', 'plus élevées'], 
         'MIN':['moins grand', 'moins grande', 'moins grands','moins grandes',
        'plus petit', 'plus petite', 'plus petits', 'plus petites','plus faible', 'plus faibles', 
         'min', 'minimal', 'minimale', 'minimales', 'minimaux','moins vite', 
         'moins élevé', 'moins élevée', 'moins élevés', 'moins élevées'],
        'MOYENNE': ['moyenne', 'moyen', 'moyennement']
        }
    COMPLEX_ANALYSIS = ['problème', 'problèmes', 'erreurs', 'erreur', 'danger']
    PAIRS = []
    question_for_simulation = ['quels véhicules ont', 'quel véhicule', 'quelle voiture', 'quelles voitures']
    reponse_for_simulation = 'véhicule'
    for borde in BORDE:
        for word in BORDE[borde]:
            for direct_variable in DIRECT_VARIABLE:
                for question_f_s in question_for_simulation:
                    if direct_variable !='position':
                        question= question_f_s+' '+direct_variable + ' '+ word
                        key0 = '#id#'
                        key1 = '#'+direct_variable+'#'
                        key2 = '#'+borde +'#'
                        reponse = reponse_for_simulation +' '+key0+ ' '+ key1+ ' '+ key2
                        PAIRS.append([question,reponse])

    question_for_simulation = ['', 'quel véhicule a', 'quelle voiture a', 'quelles voitures ont', 'il y a']
    reponse_for_simulation = 'véhicule' 
    for direct_variable in DIRECT_VARIABLE:
        for complex_analysis in COMPLEX_ANALYSIS:
            for question_f_s in question_for_simulation:
                #if direct_variable !='position':
                      question= question_f_s+' '+complex_analysis + ' '+ direct_variable
                      key0 = '#id#'
                      key1 = '#'+direct_variable+'#'
                      key2 = '#'+ complex_analysis +'#'
                      reponse = reponse_for_simulation +' '+key0+ ' '+ key2 +' '+ key1
                      PAIRS.append([question,reponse])
    return PAIRS


def clear_pairs_trains(pairs_trains):
    questions = []
    index = []
    for i in range(len(pairs_trains)):
        if  pairs_trains[i][0] not in questions:
            index.append(i)
            questions.append(pairs_trains[i][0])
    return [pairs_trains[i] for i in index]

def tf_idf(word, doc, méthode, QUESTION_TO_TRAINS):
    
    if méthode  == 'normal':
        tf, df =0, 0
        for token in doc.split():
            if word==token:
                tf+=1
        for doc_ex in QUESTION_TO_TRAINS:
            for  token  in doc_ex.split():
                if token == word:
                    df+=1
                    break
        if df>0:
            return tf*math.log(len(QUESTION_TO_TRAINS)/(df), 10)
        else:
            return 0
    if méthode == 'probabiliste':
        tf, df =0, 0
        for token in doc.split():
            if word==token:
                tf+=1
        for doc_ex in QUESTION_TO_TRAINS:
            for  token  in doc_ex.split():
                if token == word:
                    df+=1
                    break
        if df>0:
            return tf*math.log((len(QUESTION_TO_TRAINS)-df)/(df), 10)
        else:
            return 0
    if méthode == 'lissé':
        tf, df =0, 0
        for token in doc.split():
            if word==token:
                tf+=1
        for doc_ex in QUESTION_TO_TRAINS:
            for  token  in doc_ex.split():
                if token == word:
                    df+=1
                    break
        if df>0:
            return tf*(1+math.log(len(QUESTION_TO_TRAINS)/df, 10))
        else:
            return tf
    if méthode == 'probabiliste_lissé':
        tf, df =0, 0
        for token in doc.split():
            if word==token:
                tf+=1
        for doc_ex in QUESTION_TO_TRAINS:
            for  token  in doc_ex.split():
                if token == word:
                    df+=1
                    break
        if df>0:
            return tf*(1+math.log((len(QUESTION_TO_TRAINS)-df)/(df), 10))
        else:
            return tf
    if méthode =='Lucene':
        tf, df =0, 0
        for token in doc.split():
            if word==token:
                tf+=1
        for doc_ex in QUESTION_TO_TRAINS:
            for  token  in doc_ex.split():
                if token == word:
                    df+=1
                    break
        idf = 1 + math.log(len(QUESTION_TO_TRAINS)/(df+1), 10)
        tf = tf/len(QUESTION_TO_TRAINS)
        return tf, idf
    if méthode =='count':
        count_word = 0
        if word not in doc.split():
            return 0
        else:
            for doc_ex in QUESTION_TO_TRAINS:
                if word in doc_ex.split():
                    count_word +=1
            return count_word/len(QUESTION_TO_TRAINS)
    if méthode == 'inverse_tfidf':
        tf, idf = 0,0
        for token in doc.split():
            if token == word:
                tf+=1
        for doc_ex in QUESTION_TO_TRAINS:
            if word in doc_ex.split():
                idf+=1
        idf = math.log(len(QUESTION_TO_TRAINS)/(idf+1), 10)
        if tf*idf>0:
            return 1/(tf*idf)
        else:
            return 0
    

    
                    
        
        
def dic_doc_score(doc, méthode):
    """
    we say that a word in KEY_LIST is 2 times more important than other word"""
    
    if méthode != 'Lucene':
        return {word: tf_idf(word,doc, méthode, QUESTION_TO_TRAINS) for word in doc.split()}
    else:
        dictionnary ={}
        for word in doc.split():
            tf, idf = tf_idf(word,doc, méthode, QUESTION_TO_TRAINS)
            if word in KEY_LIST:
                dictionnary[word] = idf**2*2*tf
            else:
                dictionnary[word] =idf**2*tf
        return dictionnary

def score_by_new_doc(new_doc, doc, DICT, méthode):
    if méthode != 'Lucene':
        DICT_of_methode = DICT[méthode]
        score = 0
        for word in new_doc.split():
            if word in DICT_of_methode[doc]:
                 score += DICT_of_methode[doc][word]
        return score
    else:
        DICT_of_methode = DICT[méthode]
        score = 0
        for word in new_doc.split():
            if word in DICT_of_methode[doc]:
                 score += DICT_of_methode[doc][word]
        coord = 1.5*len([word for word in new_doc if word in doc])
        return  coord*score
    
def get_key_list(pairs_trains):
    KEY_LIST = []
    for pair in pairs_trains:
        for word in pair[1].split():
            if '#' in word and word not in KEY_LIST:
                KEY_LIST.append(word)
    return KEY_LIST
        

def find_right_index(new_doc,DICT, méthode):
    index=0
    biggest_score = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[0],DICT, méthode)
    for i in range(len(QUESTION_TO_TRAINS)):
        if score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i], DICT, méthode) >biggest_score:
            biggest_score = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i],DICT, méthode)
            index = i
    return index

def test(DICT, méthode, train =False):
    if not train:
        index_to_print = random.sample(range(len(pairs_tests)), 20)
        loss_total = 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_tests[i][1])
            loss_total +=loss
            if i in index_to_print:
                print(pairs_tests[i])
                print(pairs_trains[index][1], '  with accuracy ', 1-loss)
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_tests))
    if  train:
        loss_total = 0
        index_to_print = random.sample(range(len(pairs_trains)), 20)   
        for i in range(len(pairs_trains)):
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_trains[i][1])
            loss_total +=loss
            if loss >0:
                print(pairs_trains[i])
                print(pairs_trains[index][1], '  with accuracy ', 1-loss)
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_trains))

def test_without_print(DICT, méthode, train =False):
    if not train:
        loss_total = 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_tests[i][1])
            loss_total +=loss
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_tests))
    if  train:
        loss_total = 0
        index_to_print = random.sample(range(len(pairs_trains)), 20)   
        for i in range(len(pairs_trains)):
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_trains[i][1])
            loss_total +=loss
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_trains))
        
        
def chat(méthode):
    print('Bonjour, C\'est le bot d\'Avicen, posez vos questions, s\'il vous plaît!')
    path = 'processed/chat_tfidf.txt'
    with open(path, 'w') as f:
        while True:
            line = str(input('Vous: '))
            line = line.lower().strip()
            if len(line) !=0:
                index= find_right_index(line, DICT, méthode)
                print('bot: ', pairs_trains[index][1])
                f.write('VOUS ++++ '+line+'\n')
                f.write('BOT ++++ '+pairs_trains[index][1]+'\n')
            
            else:
                f.close()
                break 
def precision_recall(lstcomp, lstref) :
    card_intersec = 0.0 # force à utiliser la division non entière
    for t in set(lstcomp) :
        card_intersec += min(lstref.count(t), lstcomp.count(t))
    precision = card_intersec/len(lstcomp)
    rappel = card_intersec/len(lstref)
    return precision, rappel

    
        
def test_with_precision_recall(DICT, méthode, train = False):
    
    if not train:
        total_precision, total_recall = 0, 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            precision, recall = precision_recall(pairs_tests[i][1], pairs_trains[index][1])
            total_precision  += precision
            total_recall     += recall
        print('-'*80)
        total_precision = total_precision/len(pairs_tests)
        total_recall = total_recall/len(pairs_tests)      
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        print('PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} METHODE = {}'\
              .format(total_precision, total_recall,F, méthode))
    if  train:
        total_precision, total_recall = 0, 0
        for i in range(len(pairs_trains)):          
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            precision, recall = precision_recall(pairs_trains[i][1], pairs_trains[index][1])
            total_precision  += precision
            total_recall     += recall
        print('-'*80)
        total_precision = total_precision/len(pairs_trains)
        total_recall = total_recall/len(pairs_trains)
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        print('PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} METHODE = {}'\
              .format(total_precision, total_recall, F, méthode))
        
            
def dict_of_all_score(METHODE):
    """
    return score of all word in all doc"""
    DICT = {}
    for méthode in METHODE:
        dict_for_this_method ={}
        for doc in QUESTION_TO_TRAINS:
            dict_for_this_method[doc]=dic_doc_score(doc, méthode)
        DICT[méthode] = dict_for_this_method
    return DICT
        


if __name__=='__main__':
    try:
        pairs_trains
    except NameError :
        pairs1 = get_all_convos()
        #pairs2 = simulation_pairs()
        pairs_trains, pairs_tests = prepareData(pairs1)
        #pairs_trains = clear_pairs_trains(pairs_trains)
        KEY_LIST = get_key_list(pairs_trains)
        QUESTION_TO_TRAINS = []
        for pairs in pairs_trains:
            QUESTION_TO_TRAINS.append(pairs[0])
        METHODE = ['normal','probabiliste', 'lissé', 'probabiliste_lissé', 'Lucene', 'count', 'inverse_tfidf']
        DICT = dict_of_all_score(METHODE)   

In [17]:
for méthode in METHODE:
    test_with_precision_recall(DICT, méthode, train =True)

--------------------------------------------------------------------------------
PRECISION = 0.99787,  RECALL = 0.99379, F_MESURE = 0.996 METHODE = normal
--------------------------------------------------------------------------------
PRECISION = 0.99787,  RECALL = 0.99379, F_MESURE = 0.996 METHODE = probabiliste
--------------------------------------------------------------------------------
PRECISION = 0.99787,  RECALL = 0.99379, F_MESURE = 0.996 METHODE = lissé
--------------------------------------------------------------------------------
PRECISION = 0.99787,  RECALL = 0.99379, F_MESURE = 0.996 METHODE = probabiliste_lissé
--------------------------------------------------------------------------------
PRECISION = 0.99787,  RECALL = 0.99379, F_MESURE = 0.996 METHODE = Lucene
--------------------------------------------------------------------------------
PRECISION = 0.99833,  RECALL = 0.99523, F_MESURE = 0.997 METHODE = count
------------------------------------------------------

In [11]:
answer ='cette voiture nous semble immobilisé'
true_answer = 'cette voiture nous semble immobilisé'
precision_recall(answer, true_answer)

(0.1388888888888889, 0.1388888888888889)

In [14]:
'cette' in true_answer.split()

True

In [13]:
def metrics(lstcomp, lstref) :
    card_intersec = 0.0 # force à utiliser la division non entière
    for t in set(lstcomp) :
        card_intersec += min(lstref.count(t), lstcomp.count(t))
    precision = card_intersec/len(lstcomp)
    rappel = card_intersec/len(lstref)
    return (precision, rappel)
metrics(answer, true_answer)

(1.0, 1.0)

In [55]:
for méthode in METHODE:
    test_without_print(DICT, méthode, True)

--------------------------------------------------------------------------------
ACCURACY= 0.9863620981387479
--------------------------------------------------------------------------------
ACCURACY= 0.9863620981387479
--------------------------------------------------------------------------------
ACCURACY= 0.9863620981387479
--------------------------------------------------------------------------------
ACCURACY= 0.9863620981387479
--------------------------------------------------------------------------------
ACCURACY= 0.9863620981387479
--------------------------------------------------------------------------------
ACCURACY= 0.9883925549915398
--------------------------------------------------------------------------------
ACCURACY= 0.9828631858834904


In [56]:
for méthode in METHODE:
    test_without_print(DICT, méthode, False)

--------------------------------------------------------------------------------
ACCURACY= 0.4775205455197845
--------------------------------------------------------------------------------
ACCURACY= 0.4775205455197845
--------------------------------------------------------------------------------
ACCURACY= 0.4727128532120922
--------------------------------------------------------------------------------
ACCURACY= 0.4727128532120922
--------------------------------------------------------------------------------
ACCURACY= 0.47571570784538164
--------------------------------------------------------------------------------
ACCURACY= 0.4877859719125184
--------------------------------------------------------------------------------
ACCURACY= 0.49010464567082057


In [3]:
for méthode in METHODE:
    test_with_precision_recall(DICT, méthode, train = True)

--------------------------------------------------------------------------------
PRECISION = 0.99391,  RECALL = 0.99289, F_MESURE = 0.993 METHODE = normal
--------------------------------------------------------------------------------
PRECISION = 0.99391,  RECALL = 0.99289, F_MESURE = 0.993 METHODE = probabiliste
--------------------------------------------------------------------------------
PRECISION = 0.99391,  RECALL = 0.99289, F_MESURE = 0.993 METHODE = lissé
--------------------------------------------------------------------------------
PRECISION = 0.99391,  RECALL = 0.99289, F_MESURE = 0.993 METHODE = probabiliste_lissé
--------------------------------------------------------------------------------
PRECISION = 0.99391,  RECALL = 0.99289, F_MESURE = 0.993 METHODE = Lucene
--------------------------------------------------------------------------------
PRECISION = 0.99492,  RECALL = 0.99289, F_MESURE = 0.994 METHODE = count
------------------------------------------------------

In [18]:
for méthode in METHODE:
    test_with_precision_recall(DICT, méthode, train = False)

--------------------------------------------------------------------------------
PRECISION = 0.79171,  RECALL = 0.75531, F_MESURE = 0.773 METHODE = normal
--------------------------------------------------------------------------------
PRECISION = 0.79190,  RECALL = 0.75522, F_MESURE = 0.773 METHODE = probabiliste
--------------------------------------------------------------------------------
PRECISION = 0.79183,  RECALL = 0.74683, F_MESURE = 0.769 METHODE = lissé
--------------------------------------------------------------------------------
PRECISION = 0.79183,  RECALL = 0.74683, F_MESURE = 0.769 METHODE = probabiliste_lissé
--------------------------------------------------------------------------------
PRECISION = 0.79215,  RECALL = 0.75152, F_MESURE = 0.771 METHODE = Lucene
--------------------------------------------------------------------------------
PRECISION = 0.78681,  RECALL = 0.77215, F_MESURE = 0.779 METHODE = count
------------------------------------------------------

In [170]:
chat()

Bonjour, C'est le bot d'Avicen, posez vos questions, s'il vous plaît!
Vous: bonjour
bot:  bonjour
Vous: où est la voiture
bot:  la facture est disponible dans votre espace client
Vous: où est ma voiture
bot:  elle est à #position#
Vous: combien de km mon véhicule a roulé
bot:  il a parcouru #km#
Vous: véhicule vitesse le plus élevé
bot:  véhicule #id# #vitesse# #MAX#
Vous: véhicule qui a un problème
bot:  nous prenons en compte cet incident quel est le boitier concerné
Vous: problème véhicule
bot:  véhicule #id# #problème# #vitesse#
Vous: 


In [32]:
chat('count')

Bonjour, C'est le bot d'Avicen, posez vos questions, s'il vous plaît!
Vous: bonjour
bot:  bonjour
Vous: où est la voiture
bot:  la voiture a été localisé situé ici #position#
Vous: où est ma voiture
bot:  elle est à #position#
Vous: véhicule qui a un problème
bot:  véhicule #id# #problème# #vitesse#
Vous: voiture qui a un problème
bot:  véhicule #id# #problème# #km#
Vous: des problèmes de voitures
bot:  il y a #nb_véhicules#
Vous: problèmes de voitures
bot:  il y a #nb_véhicules#
Vous: 


In [59]:
chat('count')

Bonjour, C'est le bot d'Avicen, posez vos questions, s'il vous plaît!
Vous: bonjour
bot:  bonjour
Vous: bonsoir
bot:  #oui/non#
Vous: bonne nuit
bot:  #oui/non#
Vous: combien de voiture en route
bot:  il y a #nb_veh#
Vous: combien de voiture à l'arrêt
bot:  il y a #nb_veh#
Vous: voiture de vitesse maximale
bot:  véhicule #id# #vitesse# #max#
Vous: problème
bot:  #oui/non#
Vous: combien de voitures qui sont en circulation
bot:  #total_veh_movement#
Vous: 


In [62]:
chat('count')

Bonjour, C'est le bot d'Avicen, posez vos questions, s'il vous plaît!
Vous: bonjour
bot:  bonjour
Vous: voiture de vitesse minimale
bot:  véhicule #id# #vitesse# #min#
Vous: il y a des problèmes
bot:  #oui/non#
Vous: il y a des voitures à l'arrêt
bot:  je vérifie cela tout de suite
Vous: combien de voiture en circulations
bot:  le valeur de rachat de ce véhicule a été définie lors de l'achat de celui-ci voir contrat
Vous: combien de voiture à l'arrêt
bot:  le valeur de rachat de ce véhicule a été définie lors de l'achat de celui-ci voir contrat
Vous: 


In [63]:
for word in DICT['']

['#oui/non#',
 '#nb_véhicules#',
 '#date#',
 '#adresse#',
 '#tel_comptabilite#',
 '#vin#',
 '#kpi_pollution#',
 '#kpi_finance#',
 '#kpi_gestion#',
 '#numéro_firmware#',
 '#co2#',
 '#roi#',
 '#code_dtc#',
 '#priorité#',
 '#transmission#vin#',
 '#énergie#',
 '#cv#',
 '#consumption#',
 '#date_fin_contrat#',
 '#date_last_entretien#',
 '#trajet#',
 '#état_traffic#',
 '#geoloc#',
 '#date_debut_contrat#',
 '#km#',
 '#total_veh_movement#',
 '#nb_module#',
 '#lab_program#',
 '#num_telephone#',
 '#val_trust_low#',
 '#val_trust_high#',
 '#listing_vin#',
 '#vin_proxi#',
 '#nb_place#',
 '#lastname#',
 '#user#',
 '#list_module#',
 '#detail_module#',
 '#tx_dispo#',
 '#calcul_roadtrip#',
 '#map_meteo#',
 '#brand_tyre#',
 '#spec_tyre#',
 '#tpms#',
 '#list_vin_free#',
 '#nb_vehicule#',
 '#list_vin_immediat#',
 '#list_vin_morning#',
 '#list_vin_afternoon#',
 '#list_vin_tonight#',
 '#list_alert#',
 '#list_podium#',
 '#nb_instances#',
 '#list_instances#',
 '#histo_trip#',
 '#nb_passagers#',
 '#vin_seat#',


In [16]:
%matplotlib inline

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import data
import config
import time
from dateparser.search import search_dates
import pandas as pd


use_cuda = torch.cuda.is_available()

hidden_size = 20
SOS_token  = config.SOS_token
EOS_token  = config.EOS_token
MAX_LENGTH = config.MAX_LENGTH
stopwords  = config.STOPWORDS
learning_rate = config.LEARNING_RATE
teacher_forcing_ratio = 0.2
dropout = config.DROPOUT

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

def normalizeString(s):
    """
    Whith a tring s, we make it in lower case, delete \n if exists at 
    the end of string, and delete specical case ? . and !
    """
    s = s.lower().strip()
    s = re.sub(r"([,.!?\n])", r"", s)# sumprimer tous les caractères .! et ?
    #s = re.sub(r"[^a-zA-Z0-9.!?]+", r" ", s)
    return s

def readLangs():
    print("Reading lines...")
    pairs_trains, pairs_tests = [], []   
    encode_train = open(os.path.join(config.PROCESSED_PATH, "question_train.txt"), 'r')
    decode_train = open(os.path.join(config.PROCESSED_PATH, "answer_train.txt"), 'r')
    encode, decode = encode_train.readline(), decode_train.readline()
    while encode and decode:
        encode, decode = normalizeString(encode), normalizeString(decode)
        decode = 'SOS '+ decode + ' EOS'
        pairs_trains.append([encode, decode])
        encode, decode = encode_train.readline(), decode_train.readline()
    encode_train.close()
    decode_train.close()
    encode_test = open(os.path.join(config.PROCESSED_PATH, "question_test.txt"), 'r')
    decode_test = open(os.path.join(config.PROCESSED_PATH, "answer_test.txt"), 'r')
    encode, decode = encode_test.readline(), decode_test.readline()
    while encode and decode:
        encode, decode = normalizeString(encode), normalizeString(decode)
        decode = 'SOS '+ decode + ' EOS'
        pairs_tests.append([encode, decode])
        encode, decode = encode_test.readline(), decode_test.readline()
    encode_test.close()
    decode_test.close()
    return pairs_trains, pairs_tests


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 

    
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        for i in range(self.n_layers):
            output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result


class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result
        
def closetWord(word, lang):
    """
    find and return the closest word in lang
    """
    Dict = lang.word2index
    corpus = lang.index2word
    if word in Dict:
        return word
    else:
        distance = levenshtein(word, corpus[0])
        close_word = corpus[0]
        for ix in corpus:
            if levenshtein(word, corpus[ix]) <distance:
                close_word = corpus[ix]
                distance = levenshtein(word, corpus[ix])
        if distance <=1:
            return close_word
        else:
            return word
        
def normalizeSentenceInChat(sentence):
    sentence = sentence.strip().lower().split()
    s = [closetWord(word, input_lang) for word in sentence]
    return ' '.join(s)
                    

def indexesFromSentence(lang, sentence):
    #sentence = normalizeSentence
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index ]


def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1, 1))
    if use_cuda:
        return result.cuda()
    else:
        return result


def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    target_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)


def train(input_variable, target_variable, encoder, decoder, 
          encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]

            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
def get_skip_step(n_iters):
    return int(n_iters/10)

def trainIters(n_iters, plot_every=100, learning_rate=learning_rate):
    training_pairs = get_all_convos()
    encoder, decoder = restore_model()
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    skip_step = get_skip_step(n_iters)
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [variablesFromPair(random.choice(pairs_trains))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]

        loss = train(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % skip_step == 0:
            print_loss_avg = print_loss_total / skip_step
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    save_model(encoder, decoder)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    #sentence = normalizeSentenceInChat(sentence)
    input_variable = variableFromSentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))  
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        ni = int(ni)
        if ni == EOS_token:
            #decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])

        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    return decoded_words


def evaluate_randomly(n_iters=200, test=True):
    encoder, decoder = restore_model()
    if not test:
        print('TEST error of word to word on TRAIN')
        list_print_random = random.sample(range(n_iters),20 )
        total_loss = 0
        for i in range(n_iters):
            pair = random.choice(pairs_trains)
            output_words = evaluate(encoder, decoder, pair[0])
            if 'EOS' in output_words:
                l = output_words.index('EOS')+1
            else:
                l = len(output_words)+1
            reponse = ' '.join(output_words[:l])
            loss = _evaluate_by_right_word(pair[1], reponse)
            total_loss +=loss
            if i in list_print_random:
                #answer = answers_with_data(pair[0], reponse)
                print('Question: ', pair[0])
                print('Réponse: ', pair[1])
                print('Bot: {}   ACCURACY {:.1f}'.format(reponse, 1-loss))
                #if answer != reponse:
                 #   print('Réponse avec data:', answer)
                
                print('-'*50)
        print('Accuracy of good answer word to word: ', 1-total_loss/n_iters)
    else:
        n_iters = len(pairs_tests)
        total_loss = 0
        random_print_index = random.sample(range(n_iters), 20)
        for i in range(n_iters):
            pair = pairs_tests[i]
            output_words= evaluate(encoder, decoder, pair[0])
            if 'EOS' in output_words:
                l = output_words.index('EOS')+1
            else:
                l = len(output_words)+1
            reponse = ' '.join(output_words[:l])
            loss = _evaluate_by_right_word(pair[1], reponse)
            if i in random_print_index:
                print('Question: ', pair[0])
                print('Réponse: ', pair[1])
                print('Bot: {}. ACCURACY {:.1f}'.format(reponse, 1-loss))
                print('-'*50)
            total_loss +=loss
        print('Test on {}'.format(n_iters))
        print('Accuracy by percent of true words {}'.format(1-total_loss/n_iters))
 

KEYWORD_TO_FUNCTION_DICT = {'#id#':'ID Véhicule','#km#':'Distance', '#check_batterie#':'Tension Batterie',\
                            '#vitesse#':'Vitesse Moyenne'}

def answers_with_data(question, answer):
    data = analyse_reponse(question, answer)
    if data is None:
        return answer
    else:
        index = 0
        for word in answer.split():
            if "#" in word:
                try:
                    good_answer = re.sub(word, str(data[index]), answer)
                except IndexError:
                    break
                index +=1
        return good_answer
    
                
def analyse_reponse(question, answer):
    """
    arg: question is the question of user, reponse is given by chatbot

    """
    if '#' not in answer:
        return None
    else:
        parameter = []
        for key in KEYWORD_TO_FUNCTION_DICT:
            if key in answer.split():
                parameter.append(key)
        times = find_right_time(question)
        variables = _get_variable(answer)
        data = []
        if len(variables)==0:
            return None
        else:
            ids = find_ids(question)
            for variable in variables:
                data.append(_get_data_from_table(ids, variable, times))
            return data
            
                
def find_right_time(question):
    """
    cette fonction retourne période de temps dans la question"""
    dictionnary = {'cette':'1', 'ce':'1', "dernier": '1',"dernière": '1' }
    for word in dictionnary:
        question = re.sub(word, dictionnary[word], question)
    end_time = time.time()
    try:
         t = search_dates(question)
    except ZeroDivisionError:
        return None, None
    if t == None:
        return None, None
    
    else:
        if len(t)>=2:
        
            t0 = t[0][-1]
            start_time = time.mktime(t0.timetuple())
                
            t1 = t[-1][-1]
            end_time = time.mktime(t1.timetuple())
        if len(t)==1:
            t = t[0][1]
            start_time = time.mktime(t.timetuple())
    #start_time, end_time = find_good_time_in_table(start_time, end_time)
    return start_time, end_time
    

def _get_variable(reponse):
    list_variable =[]
    for word in reponse.split():
        if "#" in word:
            list_variable.append(word)
    #if variable in KEYWORD_TO_FUNCTION_DICT:
    return [KEYWORD_TO_FUNCTION_DICT[variable] for variable in list_variable
               if variable in KEYWORD_TO_FUNCTION_DICT]

    
def _get_data_from_table(ids, variable, time):
    """
    temps est un couple de début et fin
    """
    df = pd.read_csv('fleet_donnees.csv', sep = ';')
    #time1 = string_to_datetime(time[0])
    #time2 = string_to_datetime(time[1])
    if variable is None:
        return "#TO_CONSTRUCT#"
    if variable =='Tension Batterie':
        return min(df['Tension Batterie'])
    time1 = time[0]
    time2 = time[1]
    if time1 is None:
        time1 = string_to_datetime(df.iloc[0]['Debut période'])
    if time2 is None:
        time2 = string_to_datetime(df.iloc[len(df)-1]['Fin Période'])
    if ids is not None:
        try:
            df = df.loc[(df['ID Véhicule'] ==  int(ids))] 
        except ValueError:
            pass
    List = []
    for i in range(len(df)):
        if string_to_datetime(df.iloc[i]['Debut période']) >=time1 and \
           string_to_datetime(df.iloc[i]['Fin Période'])<=time2:
                List.append(i)

    list_data_to_return = df.iloc[List][variable]
    if len(list_data_to_return)>1:
        t = sum(list_string_to_float(list_data_to_return))/len(list_data_to_return)
    else:
        t= list_data_to_return
    
   # try:
       # t=sum(list_data_to_return.values[0])/len(list_to_return.values[0])
   # except AttributeError or IndexError:
   # t = sum(list_data_to_return)/len(list_data_to_return)
    return t
        
          
def find_ids(question):
    if 'tous' in question or 'toutes' in question or 'tout'in question or 'toute' in question:
        return None
    return None
    #else:
     #   print('La liste de ID des voitures:')
      #  imprimer_id()
       # return str(input('indiquez l\'identifiant de ce véhicule:'))
    

def string_to_datetime(string):
    t = search_dates(string)
    t = t[0][1]
    t = time.mktime(t.timetuple())
    return t

def list_string_to_float(List_string):
    List = []
    for string in List_string:
        if type(string)==str:
            string = re.sub(',', '.', string)
        else:
            pass
        string2float  = float(string)
        List.append(string2float)
    return List
    
def find_good_time_in_table(begin_time, end_time):
    """
    begin_time and end_time is in float type
    return right begin_time and end_time in table"""   
    import pandas as pd
    df = pd.read_csv('fleet_donnees.csv', sep = ';')
    start_time_colone = df['Debut période']
    end_time_colone = df['Fin Période']
    start_time_table = []
    for i in start_time_colone:
        time,_=find_right_time(i)
        start_time_table.append(time)
    end_time_table = []
    for i in fin_time_colone:
        time, _ = find_right_time(i)
        end_time_table.append(time)
    start = start_time_table[-1]
    for i in start_time_table:
        if begin_time <= i:
            start = i
            break
    end = end_time_table[-1]
    for i in end_time_table:
        if end <=i:
            end = i
            break
    #start = start_time_table[start_time_table==start].index[0]
    #end = fin_time_table[fin_time_table==end].index[0]
    start = start_time_table.index(start)
    end =  end_time_table.index(end)
    start = start_time_colone[start]
    end = _time_colone[end]
    return start, end    
    
    
def _evaluate_by_right_word(reponse, bonne_reponse):
    reponse = reponse.split()
    bonne_reponse = bonne_reponse.split()
    min_length = min(len(reponse), len(bonne_reponse))
    max_length = max(len(reponse), len(bonne_reponse))
    error = max_length-min_length
    for i in range(min_length):
        if reponse[i] != bonne_reponse[i]:
            error +=1
    return error/max_length      
        
def make_dir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass
def save_model(encoder, decoder):        
    make_dir(config.CHECK_POINT_PATH)
    path1= os.path.join(config.CHECK_POINT_PATH, 'encoder_simulation_2.ck')
    path2 = os.path.join(config.CHECK_POINT_PATH, 'decoder_simulation_2.ck')
    try: 
        os.remove(path1)  
        os.remove(path2)
    except FileNotFoundError:
        pass
    torch.save(encoder,path1)
    torch.save(decoder, path2)
    
memo = {}
def levenshtein(s, t):
    """
    Pour calculer la distance  Levenshtein entre 2 string s et t
    """
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    cost = 0 if s[-1] == t[-1] else 1
       
    i1 = (s[:-1], t)
    if not i1 in memo:
        memo[i1] = levenshtein(*i1)
    i2 = (s, t[:-1])
    if not i2 in memo:
        memo[i2] = levenshtein(*i2)
    i3 = (s[:-1], t[:-1])
    if not i3 in memo:
        memo[i3] = levenshtein(*i3)
    res = min([memo[i1]+1, memo[i2]+1, memo[i3]+cost])
    
    return res
    
def restore_model():
    #hidden_size = hidden_size
    path1 = os.path.join(config.CHECK_POINT_PATH, 'encoder_simulation_2.ck')
    path2 = os.path.join(config.CHECK_POINT_PATH, 'decoder_simulation_2.ck')
    if os.path.exists(path1)and os.path.exists(path2):
        print('Reading the parameters from {} and {}...'.format(path1, path2))
        encoder = torch.load(path1)
        decoder = torch.load(path2)
    else:
        print('Initializing fresh parameters...')
        encoder = EncoderRNN(input_lang.n_words, hidden_size)
        decoder = DecoderRNN(hidden_size, output_lang.n_words,1)

    if use_cuda:
        encoder = encoder.cuda()
        decoder = decoder.cuda()
    return encoder, decoder 

def imprimer_id():
    df = pd.read_csv('fleet_donnees.csv', sep = ';')   
    df = df['ID Véhicule']
    List_voiture = []
    for i in df:
        if i not in List_voiture:
            List_voiture.append(i)
    for i in List_voiture:
        print(i, end='  ')

def construct_dict(lang):
    in_file = open(os.path.join(config.PROCESSED_PATH, 'dictionnary.txt'), 'r')
    lines = []
    dicts = lang.word2index.copy()
    for line in in_file:
        line = line.strip().split('|')
        for word in dicts:
            if str(word) in line and len(word)>0:
                line.insert(0,str(word))
                lines.append(line)
                del dicts[str(word)]
                break
    return lines

def find_close_line(lines, lang, line):
    LINE = []
    for word in line.strip().split():
        if word in lang.word2index:
            LINE.append(word)
        else:
            for pharse in lines:
                if word in pharse:
                    LINE.append(pharse[0])
                    break
    return ' '.join(LINE)

def chat():
    encoder, decoder = restore_model()
    make_dir(config.CHECK_POINT_PATH)
    lines = construct_dict(input_lang)
    output_file = open(os.path.join(config.CHECK_POINT_PATH, 'convos70.txt'), 'a+')
    print('Bonjour, c\'est le Bot d\'AVICEN, Je peux vous aider? \n')
    while True:
            line = str(input('Vous: '))
            if len(line) > 0 and line[-1] == '\n':
                line = line[:-1]
            if line == '':
                break
            line = normalizeString(line)
            #transform_line  = normalizeSentenceInChat(line)
            #transform_line = find_close_line(lines, input_lang, transform_line)
           # print('LIGNE TRANFORMÉ: ', line)
            output_file.write('VOUS ++++ ' + line + '\n')
            reponse = evaluate(encoder, decoder, line)
            if 'SOS' in reponse:
                 reponse = reponse[1:] 
            if 'EOS' in reponse:
                reponse = reponse[:-1] 
            reponse = " ".join(reponse)
            output_file.write('BOT ++++ ' + reponse + '\n')
           # print('Without Data: ', reponse)
            #reponse = answers_with_data(line, reponse)
            print('Bot AVICEN: ', reponse)
            print('-'*50)
    output_file.close() 
    

def test(pairs_test):
    """
    Use test for know how our model is good
    """
    encoder, decoder = restore_model()
    total_loss = 0
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    for pair in pairs_test:
        variable = variablesFromPair(pair)
        loss = train( variable[0], variable[1], encoder, decoder, encoder_optimizer, decoder_optimizer,\
                     criterion= criterion)
        total_loss +=loss
    Length_inputs = len(pairs_test) if len(pairs_test) !=0 else 1
    return total_loss/Length_inputs



###############################################################################-----------------------


if __name__=='__main__':
    try:
        input_lang
    except NameError :
        pairs1 = get_all_convos()
        input_lang, output_lang, pairs_trains, pairs_tests = prepareData(pairs1, PAIRS)

NameError: name 'PAIRS' is not defined

In [7]:
STOPWORDS = ['de','d','le','la','l','du','d','ce','c','m','me','ma','si','t','sur'\
           'n','en','il', 'les','des','est','sont','s', 'a','y','au','un','une',\
          'on', 'nous', 'je', 'j','vous']


from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2


y_train = question_class
ch2 = SelectKBest(chi2)
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=1.0, min_df =0.0)
                                 #stop_words='english')
X_train = vectorizer.fit_transform(QUESTION_TO_TRAINS)
#X_train = ch2.fit_transform(X_train, y_train)


X_test = QUESTION_TO_TRAINS
X_test = vectorizer.fit_transform(X_test)
#X_test = ch2.transform(X_test)
results = []

def calcul_score(x,y):
    score = 0
    for i in range(len(x)):
        if x[i]==y[i]:
            score+=1
    return score/len(x)

def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time.time()
    clf.fit(X_train, y_train)
    train_time = time.time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time.time()
    pred = clf.predict(X_test)
    #pred = pred[0]
    #print(pairs_trains[pred])
    test_time = time.time() - t0
    print("test time:  %0.3fs" % test_time)
    score = calcul_score(y_train, pred)
    print("accuracy:   %0.3f" % score)
for clf, name in (
        (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge Classifier"),
        (Perceptron(n_iter=50), "Perceptron"),
        (PassiveAggressiveClassifier(n_iter=50), "Passive-Aggressive"),
        (KNeighborsClassifier(n_neighbors=10), "kNN"),
        (RandomForestClassifier(n_estimators=100), "Random forest")):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf))

Ridge Classifier
________________________________________________________________________________
Training: 
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='lsqr',
        tol=0.01)
train time: 0.158s
test time:  0.001s
accuracy:   0.885
Perceptron
________________________________________________________________________________
Training: 
Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=50, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:319: UserWarning: In Ridge, only 'sag' solver can currently fit the intercept when X is sparse. Solver has been automatically changed into 'sag'.
  warnings.warn("In Ridge, only 'sag' solver can currently fit the "
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


train time: 0.153s
test time:  0.001s
accuracy:   0.963
Passive-Aggressive
________________________________________________________________________________
Training: 
PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
              fit_intercept=True, loss='hinge', max_iter=None, n_iter=50,
              n_jobs=1, random_state=None, shuffle=True, tol=None,
              verbose=0, warm_start=False)
train time: 0.139s
test time:  0.001s
accuracy:   0.970
kNN
________________________________________________________________________________
Training: 
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')
train time: 0.001s
test time:  0.014s
accuracy:   0.644
Random forest
________________________________________________________________________________
Training: 
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=Non

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


train time: 0.258s
test time:  0.035s
accuracy:   0.986


In [73]:
for méthode in METHODE:
    test_without_print(DICT, méthode, False)

--------------------------------------------------------------------------------
ACCURACY= 0.37597199928660596
--------------------------------------------------------------------------------
ACCURACY= 0.37597199928660596
--------------------------------------------------------------------------------
ACCURACY= 0.3750356696985909
--------------------------------------------------------------------------------
ACCURACY= 0.3750356696985909


In [51]:
def get_all_and_all_convos():
    convos = []
    file1 = 'chatbot_tout_corpus.txt'
    file2 = 'chatbot_tout_corpus_13juin.txt'
    file3 = 'convos_120_nettoye.txt' # cette fichier viens d'etre ajouter
    #file4 = 'chatbot.txt'
    #file5 = 'convosNettoye.txt'
    #liste_file = [file1, file2, file3, file4, file5]
    liste_file = [file1, file2, file3]
    for file in liste_file:
        with open(file) as f:
            i=0
            for line in f:
                if i%2==0:
                    question = clearing(line)
                    if '++++' in question:
                         question = question[9:]
                else:
                    answer = clearing(line)
                    if '++++' in answer:
                        answer = answer[9:]
                    convos.append([question, answer])
                i+=1
        f.close()
    convos2 = simulation_pairs()
    convos.extend(convos2)
    return convos

def nettoye_all_convos(all_convos):
    """
    Supprimer tous les couples où les questions sont les mêmes"""
    index_of_convos = []
    question = []
    index =0
    for pair in all_convos:
        if pair[0] not in question:
            index_of_convos.append(index)
            question.append(pair[0])
        index +=1
    convos = [all_convos[index] for index in index_of_convos]
    print('nous avons sumpprimer {} couples parmis {} couples'.format(len(all_convos)-len(index_of_convos),index))
    return convos
def save_convos_in_file(all_convos):
    with open('convos27juin.txt', 'w') as f:
        for pair in all_convos:
            f.write('YOU ++++ '+ pair[0]+'\n')
            f.write('BOT ++++ '+ pair[1]+'\n')
    f.close()
    
def get_only_key_in_question():
    convos = []
    file1 = 'convos27.juin.txt'
    liste_file = [file1]
    for file in liste_file:
        with open(file) as f:
            i=0
            for line in f:
                if i%2==0:
                    question = clearing(line)
                    if '++++' in question:
                         question = question[9:]
                else:
                    answer = clearing(line)
                    if '++++' in answer:
                        answer = answer[9:]
                    convos.append([question, answer])
                i+=1
        f.close()
    

    

In [45]:
len(all_convos)

1506

In [52]:
all_convos = nettoye_all_convos(all_convos)
save_convos_in_file(all_convos)

nous avons sumpprimer 274 couples parmis 1506 couples
